# 复合层

到此为止，我们已经完成了我们的神经网络训练框架的核心架构，但是依然是最简单的单层网络。

我们将通过**复合层**（Composite Layer）的概念，来实现对多层网络的支持。

复合层是指把数个层组合在一起，共同完成某个复杂逻辑。同时对外提供层的接口，可以被当作一个层来使用。

在复合层内部，数个层可以是依次顺序执行，也可以是某种循环、或者判断的逻辑。

In [20]:
from abc import abstractmethod, ABC
import numpy as np

np.random.seed(99)

## 基础架构

### 张量

In [21]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = np.zeros_like(self.data)
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    @property
    def size(self):
        return self.data.shape[-1]

    def __repr__(self):
        return f'Tensor({self.data})'

### 基础数据集

In [22]:
class Dataset(ABC):

    def __init__(self, batch_size=1):
        self.batch_size = batch_size
        self.load()
        self.train()

    @abstractmethod
    def load(self):
        pass

    def train(self):
        self.features = self.train_features
        self.labels = self.train_labels

    def eval(self):
        self.features = self.test_features
        self.labels = self.test_labels

    @property
    def shape(self):
        return Tensor(self.features).size, Tensor(self.labels).size

    def items(self):
        return Tensor(self.features), Tensor(self.labels)

    def __len__(self):
        return len(self.features) // self.batch_size

    def __getitem__(self, index):
        start = index * self.batch_size
        end = start + self.batch_size

        feature = Tensor(self.features[start: end])
        label = Tensor(self.labels[start: end])
        return feature, label

### 基础层

In [23]:
class Layer(ABC):

    def __call__(self, x: Tensor):
        return self.forward(x)

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    @property
    def parameters(self):
        return []

    def __repr__(self):
        return ''

### 基础损失函数

In [24]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

### 基础优化器

In [25]:
class Optimizer(ABC):

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def reset(self):
        for p in self.parameters:
            p.grad = np.zeros_like(p.data)

    @abstractmethod
    def step(self):
        pass

### 基础模型

In [26]:
class Model(ABC):

    def __init__(self, layer, loss_fn, optimizer):
        self.layer = layer
        self.loss_fn = loss_fn
        self.optimizer = optimizer

    @abstractmethod
    def train(self, dataset, epochs):
        pass

    @abstractmethod
    def test(self, dataset):
        pass

## 数据

### 数据集

In [27]:
class Dataset(Dataset):

    def load(self):
        self.train_features = [[22.5, 72.0],
                               [31.4, 45.0],
                               [19.8, 85.0],
                               [27.6, 63.0]]
        self.train_labels = [[95],
                             [210],
                             [70],
                             [155]]

        self.test_features = [[28.1, 58.0]]
        self.test_labels = [[165]]

## 模型

### 线性层

In [28]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        self.weight = Tensor(np.random.rand(out_size, in_size) / in_size)
        self.bias = Tensor(np.random.rand(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad += p.grad.T @ x.data
            self.bias.grad += np.sum(p.grad, axis=0)
            x.grad += p.grad @ self.weight.data

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    @property
    def parameters(self):
        return [self.weight, self.bias]

    def __repr__(self):
        return f'Linear[weight{self.weight.data.shape}; bias{self.bias.data.shape}]'

### 顺序层

**顺序层**（Sequential Layer）是我们的第一个复合层，也是最简单的一种：顺序执行每一个子层。

创建一个顺序层，我们需要提供**子层列表**。

顺序层的每个子层的输出值，是下一个子层的输入值；最后一个子层的输出值，就是顺序层的输出值。

我们可以把顺序层理解成一个单纯的子层的容器。它本身没有任何参数，所以不需要额外的**梯度计算函数**（gradient_fn）和**父节点列表**（parents）。梯度计算链路在各子层之间完成传递。

但是顺序层需要把所有子层的**参数列表**（parameters）都收集起来，提供给优化器使用。

In [29]:
class Sequential(Layer):

    def __init__(self, layers):
        self.layers = layers

    def forward(self, x: Tensor):
        for l in self.layers:
            x = l(x)
        return x

    @property
    def parameters(self):
        return [p for l in self.layers for p in l.parameters]

    def __repr__(self):
        return '\n'.join(str(l) for l in self.layers if str(l))

### 损失函数（均方误差）

In [30]:
class MSELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        mse = Tensor(np.mean(np.square(y.data - p.data)))

        def gradient_fn():
            p.grad += -2 * (y.data - p.data) / len(y.data)

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

### 优化器（随机梯度下降）

In [31]:
class SGDOptimizer(Optimizer):

    def step(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

### 神经网络模型

In [32]:
class NNModel(Model):

    def train(self, dataset, epochs):
        dataset.train()

        for epoch in range(epochs):
            for i in range(len(dataset)):
                features, labels = dataset[i]

                predictions = self.layer(features)
                loss = self.loss_fn(predictions, labels)

                self.optimizer.reset()
                loss.backward()
                self.optimizer.step()

    def test(self, dataset):
        dataset.eval()

        features, labels = dataset.items()
        predictions = self.layer(features)
        loss = self.loss_fn(predictions, labels)
        return predictions, loss

## 设置

### 学习率

In [33]:
LEARNING_RATE = 0.00001

### 批大小

In [34]:
BATCH_SIZE = 2

### 轮次

In [35]:
EPOCHS = 1000

## 训练

### 建模

通过使用顺序层，我们能轻松实现多层神经网络模型。

需要注意的，是对齐上一层输出值的数量和下一层输入值的数量。在我们的例子中，隐藏层输出 4 个中间值，所以输出层有 4 个输入值。

In [36]:
dataset = Dataset(BATCH_SIZE)

layer = Sequential([Linear(dataset.shape[0], 4),
                    Linear(4, dataset.shape[1])])
loss_fn = MSELoss()
optimizer = SGDOptimizer(layer.parameters, lr=LEARNING_RATE)

model = NNModel(layer, loss_fn, optimizer)
print(model.layer)

Linear[weight(4, 2); bias(4,)]
Linear[weight(1, 4); bias(1,)]


### 迭代

In [37]:
model.train(dataset, EPOCHS)

## 验证

### 测试

In [38]:
predictions, loss = model.test(dataset)
print(f'prediction: {predictions}')
print(f'loss: {loss}')

prediction: Tensor([[166.58568956]])
loss: Tensor(2.5144113737205855)


使用我们的神经网络训练框架，实现多层网络模型是再简单不过的事情了。

## 课后练习

1. 调整隐藏层输出的中间值数量，观察对最终结果的影响。
2. 尝试增加更多的隐藏层，观察对最终结果的影响。
3. 试一试，能不能把顺序层改装成一个循环层？